<a href="https://colab.research.google.com/github/Maanas-Verma/cerebrotest/blob/master/code_in%20_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


In [0]:
import pandas as pd
# Importing the dataset
dataset = pd.read_csv('train.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X=np.append(arr=np.ones((7192,1)).astype(int),values=X,axis=1)



In [0]:
X_opt=X[:,[2,3,19,21]]  

In [4]:
import statsmodels.api as sm

X_opt_1=np.append(arr=np.ones((7192,1)).astype(int),values=X_opt,axis=1)
regressor_OLS=sm.OLS(endog=y,exog=X_opt_1).fit()
regressor_OLS.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.806
Date:                Sun, 13 Oct 2019   Prob (F-statistic):            0.00971
Time:                        14:10:25   Log-Likelihood:                -5178.5
No. Observations:                7192   AIC:                         1.036e+04
Df Residuals:                    7188   BIC:                         1.039e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1519      0.016      9.293      0.000       0.120       0.184
x1          2.899e-06   1.24e-06      2.344      0.019    4.75e-07    5.32e-06
x2          2.899e-06   1.24e-06      2.344      0.019    4.75e-07    5.32e-06
x3            -0.0002      0.000     -1.673      0.094      -0.000    3.87e-05
x4             0.0130      0.007      1.809      0.070      -0.001       0.027
==============================================================================
Omnibus:                        6.373   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                6.384
Skew:                           0.066   Prob(JB):                       0.0411
Kurtosis:                       2.936   Cond. No.                     2.63e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.67e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [0]:
NN_model = Sequential()

In [0]:
NN_model.add(Dense(4, kernel_initializer='normal',input_dim = X_opt.shape[1], activation='elu'))

In [0]:
NN_model.add(Dense(200, kernel_initializer='normal',activation='elu'))
NN_model.add(Dense(100, kernel_initializer='normal',activation='elu'))
NN_model.add(Dense(100, kernel_initializer='normal',activation='elu'))

In [0]:
NN_model.add(Dense(1, kernel_initializer='normal',activation='elu'))

In [40]:
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_14 (Dense)             (None, 20)                100       
_________________________________________________________________
dense_15 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 21        
Total params: 561
Trainable params: 561
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [41]:
NN_model.fit(X_opt, y, epochs=500, batch_size=23, validation_split = 0.4, callbacks=callbacks_list)

Train on 4315 samples, validate on 2877 samples
Epoch 1/500
4315/4315 [==============================] - 1s 207us/step - loss: 0.2478 - mean_squared_error: 0.2478 - val_loss: 0.2537 - val_mean_squared_error: 0.2537

Epoch 00001: val_loss did not improve from 0.25254
Epoch 2/500
4315/4315 [==============================] - 0s 76us/step - loss: 0.2443 - mean_squared_error: 0.2443 - val_loss: 0.2530 - val_mean_squared_error: 0.2530

Epoch 00002: val_loss did not improve from 0.25254
Epoch 3/500
4315/4315 [==============================] - 0s 75us/step - loss: 0.2436 - mean_squared_error: 0.2436 - val_loss: 0.2527 - val_mean_squared_error: 0.2527

Epoch 00003: val_loss did not improve from 0.25254
Epoch 4/500
4315/4315 [==============================] - 0s 74us/step - loss: 0.2441 - mean_squared_error: 0.2441 - val_loss: 0.2528 - val_mean_squared_error: 0.2528

Epoch 00004: val_loss did not improve from 0.25254
Epoch 5/500
4315/4315 [==============================] - 0s 71us/step - loss: 0

In [0]:
# Load wights file of the best model :
wights_file = 'Weights-001--0.34596.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [0]:
def make_submission(prediction, sub_name):
  my_submission = pd.DataFrame({'id':pd.read_csv('test.csv').id,'target':prediction})
  my_submission.to_csv('{}.csv'.format(sub_name),index=True)
  print('A submission file has been made')

In [0]:
dataset_t = pd.read_csv('test.csv')
dataset_t = np.append(arr=np.ones((1000,1)).astype(int),values=dataset_t,axis=1)
data_t=dataset_t[:,[2,3,19,21]]  


In [0]:
predictions = NN_model.predict(data_t)

In [35]:
make_submission(predictions[:,0],'submission(annsmall2)4f.csv')

A submission file has been made


In [93]:
%ls


 sample_data/                Weights-009--0.24531.hdf5
'submission(aNN).csv.csv'    Weights-009--0.24899.hdf5
'submission(NN).csv.csv'     Weights-010--0.24382.hdf5
 test.csv                    Weights-012--0.24480.hdf5
 train.csv                   Weights-013--0.24098.hdf5
 Weights-001--0.28361.hdf5   Weights-014--0.24510.hdf5
 Weights-001--0.28611.hdf5   Weights-015--0.24128.hdf5
 Weights-001--0.28653.hdf5   Weights-018--0.24251.hdf5
 Weights-001--0.34596.hdf5   Weights-019--0.24369.hdf5
 Weights-002--0.25055.hdf5   Weights-020--0.24179.hdf5
 Weights-002--0.25616.hdf5   Weights-021--0.24193.hdf5
 Weights-002--0.26218.hdf5   Weights-022--0.24113.hdf5
 Weights-003--0.24439.hdf5   Weights-023--0.24159.hdf5
 Weights-004--0.24758.hdf5   Weights-029--0.24037.hdf5
 Weights-005--0.24740.hdf5   Weights-069--0.24158.hdf5
 Weights-007--0.24456.hdf5
